# Проект по SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. На какое-то время жители городов перестали выходить на улицу, посещать кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать. Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке.

**Цель** — проанализировать базу данных и сформулировать ценностное предложение для нового продукта.

**Описание данных:**

Таблица books - cодержит данные о книгах:
+ book_id — идентификатор книги;
+ author_id — идентификатор автора;
+ title — название книги;
+ num_pages — количество страниц;
+ publication_date — дата публикации книги;
+ publisher_id — идентификатор издателя.

Таблица authors - cодержит данные об авторах:
+ author_id — идентификатор автора;
+ author — имя автора.

Таблица publishers - cодержит данные об издательствах:
+ publisher_id — идентификатор издательства;
+ publisher — название издательства.

Таблица ratings - cодержит данные о пользовательских оценках книг:
+ rating_id — идентификатор оценки;
+ book_id — идентификатор книги;
+ username — имя пользователя, оставившего оценку;
+ rating — оценка книги.

Таблица reviews - cодержит данные о пользовательских обзорах:
+ review_id — идентификатор обзора;
+ book_id — идентификатор книги;
+ username — имя автора обзора;
+ text — текст обзора.

**Задачи:**

+ Посчитать, сколько книг вышло после 1 января 2000 года;
+ Для каждой книги посчитать количество обзоров и среднюю оценку;
+ Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;
+ Определить автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками;
+ Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [ ]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [ ]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [ ]:
# Функция для выполнения SQL-запрос, используем Pandas

def test(query):
    con = engine.connect()
    result = pd.io.sql.read_sql(sql=text(query), con=con)
    return result

In [ ]:
# Посмотрим данные из таблицы books
test('''SELECT *
        FROM books
        LIMIT 1
        ''')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


In [ ]:
# Посмотрим данные из таблицы authors
test('''SELECT *
        FROM authors
        LIMIT 1
        ''')

,author_id,author
0,1,A.S. Byatt


In [ ]:
# Посмотрим данные из таблицы publishers
test('''SELECT *
        FROM publishers
        LIMIT 1
        ''')

,publisher_id,publisher
0,1,Ace


In [ ]:
# Посмотрим данные из таблицы ratings
test('''SELECT *
        FROM ratings
        LIMIT 1
        ''')

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4


In [ ]:
# Посмотрим данные из таблицы reviews
test('''SELECT *
        FROM reviews
        LIMIT 1
        ''')

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


<div style="border:solid grey 2px; padding: 40px">
Посмотрели данные. Данные соответствуют описанию и обозначенной структуре.
Данных достаточно для анализа.

In [ ]:
# Выгрузим количество книг, которое вышло после 1 января 2000 года
test('''SELECT COUNT(book_id)
        FROM books
        WHERE CAST(publication_date AS date)>'2000-01-01'
        ''')

,count
0,819


<div style="border:solid grey 2px; padding: 40px">
819 книг вышло после 1 января 2000 года.

In [ ]:
# Посчитаем количество обзоров  и среднюю оценку для каждой книги
test('''SELECT b.title,
               COUNT(DISTINCT r.review_id),
               ROUND(AVG(rat.rating),2)
        FROM books AS b
        LEFT JOIN reviews AS r ON b.book_id=r.book_id
        LEFT JOIN ratings AS rat ON rat.book_id=b.book_id
        GROUP BY b.book_id
        ORDER BY COUNT(DISTINCT r.review_id) DESC;
        ''')

,title,count,round
0,Twilight (Twilight #1),7,3.66
1,Water for Elephants,6,3.98
2,The Glass Castle,6,4.21
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
4,The Curious Incident of the Dog in the Night-Time,6,4.08
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67
996,The Natural Way to Draw,0,3.00
997,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
998,Essential Tales and Poems,0,4.00


<div style="border:solid grey 2px; padding: 40px">

Для каждой книги вывели количество обзоров и среднюю оценку. Максимальное количество уникальных обзоров - 7, минимальное - 0. Рейтинг вариьруется от 1.5 до 5.00.

In [ ]:
# Определим издательство, которое выпустило наибольшее чило книг толще 50 страниц
test('''SELECT p.publisher,
               COUNT(b.book_id)
        FROM books AS b
        JOIN publishers AS p ON b.publisher_id=p.publisher_id
        GROUP BY p.publisher
        ORDER BY COUNT(b.book_id) DESC
        LIMIT 1;
        ''')

,publisher,count
0,Penguin Books,42


<div style="border:solid grey 2px; padding: 40px">
Наибольшее число книг толще 50 страниц выпустило издательство Penguin Books.

In [ ]:
# Определим автора с самой высокой средней оценкой книг, учитываем только книги с 50 и более оценками.

test('''with rat AS (
                        SELECT b.book_id,
                               COUNT(rating_id) AS cnt_rating,
                               AVG(rating) AS avg_rating
                        FROM ratings AS r
                        JOIN books AS b ON r.book_id=b.book_id
                        GROUP BY b.book_id
                        )

          SELECT a.author,
                 ROUND(AVG(avg_rating),2)
          FROM authors as a
          JOIN books AS b ON b.author_id = a.author_id
          JOIN rat ON rat.book_id = b.book_id
          WHERE rat.cnt_rating >= 50
          GROUP BY a.author
          ORDER BY AVG(avg_rating) DESC
          LIMIT 1''')

,author,round
0,J.K. Rowling/Mary GrandPré,4.28


<div style="border:solid grey 2px; padding: 40px">
J.K. Rowling/Mary GrandPré - автор с самой высокой средней оценкой книг, при условии, что мы учитывали книги с 50 и более оценками.

In [ ]:
# Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

test('''with rat AS (
                     SELECT username
                     FROM ratings
                     GROUP BY username
                     HAVING COUNT(*) > 48
                     )
SELECT AVG(cnt_reviews)
FROM (
       SELECT r.username, COUNT(*) AS cnt_reviews
       FROM reviews r
       INNER JOIN rat ON r.username = rat.username
       GROUP BY r.username
) AS rev;''')

,avg
0,24.0


<div style="border:solid grey 2px; padding: 40px">
В среднем 24 обзора оставляют пользователи, которые поставили более 48 оценок.

<div style="border:solid green 2px; padding: 40px">
    
**ВЫВОД**

В базе крупного сервиса для чтения книг по подписке - 1000 книг. Есть отзывы на каждую книгу и оценка. Конечно, отзывов меньше, чем просто оценок, поскольку оценка занимает гораздо меньше времени. После 1 января 2000 вышло 819 книг. Представлены различные издательства. Есть книги популярных авторов, например, Дж. К. Роулинг. Благодаря всем перечисленным плюсам удастся удовлетворить запросы многих читателей.